# example of using the ColumnTransformer for the Abalone dataset

In [ ]:
# load dataset
from pandas import read_csv
dataframe = read_csv('abalone.csv', header=None)

In [ ]:
# split into inputs and outputs
last_ix = len(dataframe.columns) - 1
X, y = dataframe.drop(last_ix, axis=1), dataframe[last_ix]
print(X.shape, y.shape)

In [ ]:
# determine categorical and numerical features
numerical_ix = X.select_dtypes(include=['int64', 'float64']).columns
categorical_ix = X.select_dtypes(include=['object', 'bool']).columns

In [ ]:
# define the data preparation for the columns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
t = [('cat', OneHotEncoder(), categorical_ix), ('num', MinMaxScaler(), numerical_ix)]
col_transform = ColumnTransformer(transformers=t)

In [ ]:
# define the model
from sklearn.svm import SVR
model = SVR(kernel='rbf',gamma='scale',C=100)

In [ ]:
# define the data preparation and modeling pipeline
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps=[('prep',col_transform), ('m', model)])

In [ ]:
# define the model cross-validation configuration
from sklearn.model_selection import KFold
cv = KFold(n_splits=10, shuffle=True, random_state=1)

In [ ]:
# evaluate the pipeline using cross validation and calculate MAE
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipeline, X, y, scoring='neg_mean_absolute_error', cv=cv,
n_jobs=-1)

In [ ]:
# convert MAE scores to positive values
from numpy import absolute
scores = absolute(scores)

In [ ]:
# summarize the model performance
from numpy import mean
from numpy import std
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

# example of normalizing input and output variables for regression.

In [ ]:
# load data
from numpy import loadtxt
dataset = loadtxt('housing.csv', delimiter=",")

In [ ]:
# split into inputs and outputs
X, y = dataset[:, :-1], dataset[:, -1]

In [ ]:
# prepare the model with input scaling
from sklearn.linear_model import HuberRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps=[('normalize', MinMaxScaler()), ('model', HuberRegressor())])

In [ ]:
# prepare the model with target scaling
from sklearn.compose import TransformedTargetRegressor
model = TransformedTargetRegressor(regressor=pipeline, transformer=MinMaxScaler())

In [ ]:
# evaluate model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

In [ ]:
# convert scores to positive
from numpy import absolute
scores = absolute(scores)

In [ ]:
# summarize the result
from numpy import mean
s_mean = mean(scores)
print('Mean MAE: %.3f' % (s_mean))

# example of power transform input and output variables for regression.

In [ ]:
# load data
from numpy import loadtxt
dataset = loadtxt('housing.csv', delimiter=",")

In [ ]:
# split into inputs and outputs
X, y = dataset[:, :-1], dataset[:, -1]

In [ ]:
# prepare the model with input scaling and power transform
from sklearn.linear_model import HuberRegressor
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
steps = list()
steps.append(('scale', MinMaxScaler(feature_range=(1e-5,1))))
steps.append(('power', PowerTransformer()))
steps.append(('model', HuberRegressor()))
pipeline = Pipeline(steps=steps)

In [ ]:
# prepare the model with target scaling
from sklearn.compose import TransformedTargetRegressor
model = TransformedTargetRegressor(regressor=pipeline, transformer=PowerTransformer())

In [ ]:
# evaluate model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

In [ ]:
# convert scores to positive
from numpy import absolute
scores = absolute(scores)

In [ ]:
# summarize the result
from numpy import mean
s_mean = mean(scores)
print('Mean MAE: %.3f' % (s_mean))

# example of creating a test dataset and splitting it into train and test sets

In [ ]:
# prepare dataset
from sklearn.datasets import make_blobs
X, y = make_blobs(n_samples=100, centers=2, n_features=2, random_state=1)

In [ ]:
# split data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [ ]:
# summarize the scale of each input variable
for i in range(X_test.shape[1]):
    print('>%d, train: min=%.3f, max=%.3f, test: min=%.3f, max=%.3f' %
        (i, X_train[:, i].min(), X_train[:, i].max(),
            X_test[:, i].min(), X_test[:, i].max()))

# example of scaling the dataset

In [ ]:
# prepare dataset
from sklearn.datasets import make_blobs
X, y = make_blobs(n_samples=100, centers=2, n_features=2, random_state=1)

In [ ]:
# split data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [ ]:
# define scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
# fit scaler on the training dataset
scaler.fit(X_train)

In [ ]:
# transform both datasets
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# summarize the scale of each input variable
for i in range(X_test.shape[1]):
    print('>%d, train: min=%.3f, max=%.3f, test: min=%.3f, max=%.3f' %
        (i, X_train_scaled[:, i].min(), X_train_scaled[:, i].max(),
            X_test_scaled[:, i].min(), X_test_scaled[:, i].max()))

# example of fitting a model on the scaled dataset

In [ ]:
# prepare dataset
from sklearn.datasets import make_blobs
X, y = make_blobs(n_samples=100, centers=2, n_features=2, random_state=1)

In [ ]:
# split data into train and test sets
from sklearn.model_selection import train_test_split
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.33, random_state=1)

In [ ]:
# define scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
# fit scaler on the training dataset
scaler.fit(X_train)

In [ ]:
# transform the training dataset
X_train_scaled = scaler.transform(X_train)

In [ ]:
# define model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs')
model.fit(X_train_scaled, y_train)

In [ ]:
# save the model
from pickle import dump
dump(model, open('model.pkl', 'wb'))

In [ ]:
# save the scaler
from pickle import dump
dump(scaler, open('scaler.pkl', 'wb'))

# load model and scaler and make predictions on new data

In [ ]:
# prepare dataset
from sklearn.datasets import make_blobs
X, y = make_blobs(n_samples=100, centers=2, n_features=2, random_state=1)

In [ ]:
# split data into train and test sets
from sklearn.model_selection import train_test_split
_, X_test, _, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [ ]:
# load the model
from pickle import load
model = load(open('model.pkl', 'rb'))

In [ ]:
# load the scaler
from pickle import load
scaler = load(open('scaler.pkl', 'rb'))

In [ ]:
# check scale of the test set before scaling
print('Raw test set range')
for i in range(X_test.shape[1]):
    print('>%d, min=%.3f, max=%.3f' % (i, X_test[:, i].min(), X_test[:, i].max()))

In [ ]:
# transform the test dataset
X_test_scaled = scaler.transform(X_test)
print('Scaled test set range')
for i in range(X_test_scaled.shape[1]):
    print('>%d, min=%.3f, max=%.3f' % (i, X_test_scaled[:, i].min(), X_test_scaled[:,
        i].max()))

In [ ]:
# evaluate accuracy
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, yhat)
print('Test Accuracy:', acc)